In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import imutils
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import math
from sklearn.metrics import f1_score
#pip install opencv-python pandas easyocr

import imutils
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers

from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Input, Dropout
from keras.models import Model, Sequential
#from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [ ]:
# Define the folder path
folder_path = 'C:/Users/Dell/JupyterPythoncodes/DataScienceSoulpageit assignment/license_plates_detection_train'

# Get a list of all files in the folder
image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Loop through each image file
for image_file in image_files:
    # Read the image
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)

    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    image = imutils.resize(image, width=500)
    img=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the original image
    fig, ax = plt.subplots(2, 2, figsize=(10,7))
    ax[0,0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    ax[0,0].set_title('Original Image')

    # RGB to Gray scale conversion
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ax[0,1].imshow(gray, cmap='gray')
    ax[0,1].set_title('Grayscale Conversion')

    # Noise removal with iterative bilateral filter(removes noise while preserving edges)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    ax[1,0].imshow(gray, cmap='gray')
    ax[1,0].set_title('Bilateral Filter')

    # Find Edges of the grayscale image
    edged = cv2.Canny(gray, 170, 200)
    ax[1,1].imshow(edged, cmap='gray')
    ax[1,1].set_title('Canny Edges')

    fig.tight_layout()
    plt.show()
    
    # Find contours based on Edges
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
    cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30] #sort contours based on their area keeping minimum required area as '30' (anything smaller than this will not be considered)
    NumberPlateCnt = None #we currently have no Number plate contour

    # loop over our contours to find the best possible approximate contour of number plate
    count = 0
    for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            if len(approx) == 4:  # Select the contour with 4 corners
                NumberPlateCnt = approx #This is our approx Number Plate Contour
                x,y,w,h = cv2.boundingRect(c)
                ROI = img[y:y+h, x:x+w]
                break

    if NumberPlateCnt is not None:
        # Drawing the selected contour on the original image
        cv2.drawContours(image, [NumberPlateCnt], -1, (0,255,0), 3)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title("Detected license plate")
        plt.show()

        # Find bounding box and extract ROI
        plt.imshow(ROI)
        plt.title("Extracted license plate")
        plt.show()
        
        print(NumberPlateCnt) #if the extracted license plate is tilted/rotated

        # Distance between (x1, y1) and (x2, y2)
        def dist(x1, x2, y1, y2):
            return ((x1-x2)**2+(y1-y2)**2)**0.5

        idx=0
        m=0
        # To find the index of coordinate with maximum y-coordinate
        for i in range(4):
            if NumberPlateCnt[i][0][1]>m:
                idx=i
                m=NumberPlateCnt[i][0][1]

        # Assign index to the previous coordinate
        if idx==0:
            pin=3
        else:
            pin=idx-1

        # Assign index to the next coordinate
        if idx==3:
            nin=0
        else:
            nin=idx+1

        # Find distances between the acquired coordinate and its previous and next coordinate
        p=dist(NumberPlateCnt[idx][0][0], NumberPlateCnt[pin][0][0], NumberPlateCnt[idx][0][1], NumberPlateCnt[pin][0][1])
        n=dist(NumberPlateCnt[idx][0][0], NumberPlateCnt[nin][0][0], NumberPlateCnt[idx][0][1], NumberPlateCnt[nin][0][1])

        # The coordinate that has more distance from the acquired coordinate is the required second bottom-most coordinate
        if p>n:
            if NumberPlateCnt[pin][0][0]<NumberPlateCnt[idx][0][0]:
                left=pin
                right=idx
            else:
                left=idx
                right=pin
            d=p
        else:
            if NumberPlateCnt[nin][0][0]<NumberPlateCnt[idx][0][0]:
                left=nin
                right=idx
            else:
                left=idx
                right=nin
            d=n
        print(left, right)
        left_x=NumberPlateCnt[left][0][0]
        left_y=NumberPlateCnt[left][0][1]
        right_x=NumberPlateCnt[right][0][0]
        right_y=NumberPlateCnt[right][0][1]
        print(left_x, left_y, right_x, right_y)

        # Finding the angle of rotation by calculating sin of theta
        opp=right_y-left_y
        hyp=((left_x-right_x)**2+(left_y-right_y)**2)**0.5
        sin=opp/hyp
        theta=math.asin(sin)*57.2958

        # Rotate the image according to the angle of rotation obtained
        image_center = tuple(np.array(ROI.shape[1::-1]) / 2)
        rot_mat = cv2.getRotationMatrix2D(image_center, theta, 1.0)
        result = cv2.warpAffine(ROI, rot_mat, ROI.shape[1::-1], flags=cv2.INTER_LINEAR)

        # The image can be cropped after rotation( since rotated image takes much more height)
        if opp>0:
            h=result.shape[0]-opp//2
        else:
            h=result.shape[0]+opp//2

        result=result[0:h, :]
        plt.imshow(result)
        plt.title("Plate obtained after rotation")
        plt.show()


In [ ]:
# Data preprocessing
import pandas as pd
import cv2
#import cv2_imshow
import os

# read annotation file
df = pd.read_csv("C:/Users/Dell/JupyterPythoncodes/DataScienceSoulpageit assignment/Licplatesdetection_train.csv")

print(df)

In [ ]:
import matplotlib.pyplot as plt
import os
# Set up matplotlib to display images inline in the notebook
%matplotlib inline

# Process each row in the dataframe
for index, row in df.iterrows():
    img_id = row['img_id']
    x_min, y_min, x_max, y_max = int(row["xmin"]), int(row["ymin"]), int(row["xmax"]), int(row["ymax"])
    
    # Print the bounding box coordinates
    print(f"Image ID: {img_id}, Coordinates: ({x_min}, {y_min}, {x_max}, {y_max})")
    
    # Read image
    img_path = os.path.join("C:\Users\Dell\JupyterPythoncodes\DataScienceSoulpageit assignment\license_plates_detection_train", img_id)  # Adjust directory as necessary
    img = cv2.imread(img_path)
    
    if img is not None:
        # Print image shape
        print(f"Image Shape: {img.shape}")  # Output the dimensions of the image
        
        # Crop the region containing the license plate
        plate = img[y_min:y_max, x_min:x_max]
        
        # Convert BGR image to RGB
        plate_rgb = cv2.cvtColor(plate, cv2.COLOR_BGR2RGB)
        
        # Display the cropped license plate
        plt.figure(figsize=(5,5))  # Set the figure size (optional)
        plt.imshow(plate_rgb)
        plt.axis('off')  # Turn off axis numbers and ticks
        plt.show()
    else:
        print(f"Failed to read image {img_id}")
# Process each row in the dataframe
for index, row in df.iterrows():
    img_id = row['img_id']
    x_min, y_min, x_max, y_max = int(row["xmin"]), int(row["ymin"]), int(row["xmax"]), int(row["ymax"])

    # Print the bounding box coordinates
    print(f"Image ID: {img_id}, Coordinates: ({x_min}, {y_min}, {x_max}, {y_max})")

    # Read image
    img_path = os.path.join("C:\Users\Dell\JupyterPythoncodes\DataScienceSoulpageit assignment\license_plates_detection_train", img_id)  # Adjust directory as necessary
    img = cv2.imread(img_path)

    if img is not None:
        # Print image shape
        print(f"Image Shape: {img.shape}")  # Output the dimensions of the image

        # Crop the region containing the license plate
        plate = img[y_min:y_max, x_min:x_max]

        # Show the cropped license plate
        cv2_imshow(plate)
    else:
        print(f"Failed to read image {img_id}")

In [ ]:
img_bbox = df.loc[df['img_id']=='11.jpg']
x_min, y_min, x_max, y_max = int(img_bbox["xmin"]), int(img_bbox["ymin"]), int(img_bbox["xmax"]), int(img_bbox["ymax"])
print(x_min, y_min, x_max, y_max)
# plot the bbox and visualize
img = cv2.imread("C:/Users/Dell/JupyterPythoncodes/DataScienceSoulpageit assignment/license_plates_detection_train/11.jpg")
print(img.shape) # -> (h, w , 3) ->(BGR)

# crop the region
plate = img[y_min:y_max,  x_min:x_max]
cv2_imshow(plate)
# img[]

In [ ]:
def segment_characters(image) :
    # Preprocess cropped license plate image
    img_lp = cv2.resize(image, (333, 75))
    img_gray_lp = cv2.cvtColor(img_lp, cv2.COLOR_BGR2GRAY)
    _, img_binary_lp = cv2.threshold(img_gray_lp, 200, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img_binary_lp = cv2.erode(img_binary_lp, (3,3))
    img_binary_lp = cv2.dilate(img_binary_lp, (3,3))

    LP_WIDTH = img_binary_lp.shape[0]
    LP_HEIGHT = img_binary_lp.shape[1]

    # Make borders white
    img_binary_lp[0:3,:] = 255
    img_binary_lp[:,0:3] = 255
    img_binary_lp[72:75,:] = 255
    img_binary_lp[:,330:333] = 255

    # Estimations of character contours sizes of cropped license plates
    dimensions = [LP_WIDTH/6,
                       LP_WIDTH/2,
                       LP_HEIGHT/10,
                       2*LP_HEIGHT/3]
    plt.imshow(img_binary_lp, cmap='gray')
    plt.title('Contour')
    plt.show()
    cv2.imwrite('contour.jpg',img_binary_lp)

    # Get contours within cropped license plate
    char_list = find_contours(dimensions, img_binary_lp)

    return char_list

In [ ]:
# Match contours to license plate or character template
def find_contours(dimensions, img) :

    # Find all contours in the image
    cntrs, _ = cv2.findContours(img.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Retrieve potential dimensions
    lower_width = dimensions[0]
    upper_width = dimensions[1]
    lower_height = dimensions[2]
    upper_height = dimensions[3]

    # Check largest 5 or  15 contours for license plate or character respectively
    cntrs = sorted(cntrs, key=cv2.contourArea, reverse=True)[:15]

    ii = cv2.imread('contour.jpg')

    x_cntr_list = []
    target_contours = []
    img_res = []
    for cntr in cntrs :
        # detects contour in binary image and returns the coordinates of rectangle enclosing it
        intX, intY, intWidth, intHeight = cv2.boundingRect(cntr)

        # checking the dimensions of the contour to filter out the characters by contour's size
        if intWidth > lower_width and intWidth < upper_width and intHeight > lower_height and intHeight < upper_height :
            x_cntr_list.append(intX) #stores the x coordinate of the character's contour, to used later for indexing the contours

            char_copy = np.zeros((44,24))
            # extracting each character using the enclosing rectangle's coordinates.
            char = img[intY:intY+intHeight, intX:intX+intWidth]
            char = cv2.resize(char, (20, 40))

            cv2.rectangle(ii, (intX,intY), (intWidth+intX, intY+intHeight), (50,21,200), 2)
            plt.imshow(ii, cmap='gray')
            plt.title('Predict Segments')

            # Make result formatted for classification: invert colors
            char = cv2.subtract(255, char)

            # Resize the image to 24x44 with black border
            char_copy[2:42, 2:22] = char
            char_copy[0:2, :] = 0
            char_copy[:, 0:2] = 0
            char_copy[42:44, :] = 0
            char_copy[:, 22:24] = 0

            img_res.append(char_copy) # List that stores the character's binary image (unsorted)

    # Return characters on ascending order with respect to the x-coordinate (most-left character first)

    plt.show()
    # arbitrary function that stores sorted list of character indeces
    indices = sorted(range(len(x_cntr_list)), key=lambda k: x_cntr_list[k])
    img_res_copy = []
    for idx in indices:
        img_res_copy.append(img_res[idx])# stores character images according to their index
    img_res = np.array(img_res_copy)

    return img_res

In [ ]:
segment_characters(image)